In [4]:
import cv2
from pathlib import Path as P
import pandas as pd
data_root = P('/home/zhaobinguet/codes/datasets/TianChiBuilding')
train_folder = data_root / 'train'
train_mask_csv = data_root / 'train_mask.csv'

# read the csv 
train_mask_df = pd.read_csv(train_mask_csv)


In [5]:
import numpy as np

import pandas as pd
import cv2
import matplotlib.pyplot as plt

# 将图片编码为rle格式
def rle_encode(im):
    '''
    im: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 将rle格式进行解码为图片
def rle_decode(mask_rle, shape=(512, 512)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')

def show_examples(name: str, image: np.ndarray, mask: np.ndarray):
    plt.figure(figsize=(10, 14))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title(f"Image: {name}")

    plt.subplot(1, 2, 2)
    plt.imshow(mask, cmap='gray')
    plt.title(f"Mask: {name}")
    

In [6]:
import pandas
import os
from pathlib import Path as P
from tqdm import tqdm
from PIL import Image

train_root = data_root / 'train'
mask_root = data_root / 'mask'

i = 0
csv = pandas.read_csv(os.path.join(data_root, 'train_mask.csv'), sep='\t', names=['name', 'mask'])
with tqdm(total=len(csv['name'].iloc[:])) as pbar:
    for file_name, mask_code in zip(csv['name'].iloc[:], csv['mask'].iloc[:]):
        mask_code = '' if str(mask_code) == 'nan' else mask_code
        raw_img = plt.imread(os.path.join(train_root, file_name))
        mask_img = rle_decode(mask_code)
        mask_img = Image.fromarray(mask_img * 255)
        mask_img.save(mask_root / file_name)
        pbar.update(1)


100%|██████████| 30000/30000 [01:00<00:00, 497.13it/s]
